In [11]:

import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
import warnings
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from google.colab import drive
drive.mount('/content/drive')
warnings.filterwarnings('ignore')

CSV_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/Cyber_ML/Merged01.csv'
df = pd.read_csv(CSV_FILE_PATH)

print(f"Dataset shape: {df.shape}")
print(f"Dataset info:")
print(df.info())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset shape: (712311, 40)
Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712311 entries, 0 to 712310
Data columns (total 40 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Header_Length    712311 non-null  float64
 1   Protocol Type    712311 non-null  int64  
 2   Time_To_Live     712311 non-null  float64
 3   Rate             712311 non-null  float64
 4   fin_flag_number  712311 non-null  float64
 5   syn_flag_number  712311 non-null  float64
 6   rst_flag_number  712311 non-null  float64
 7   psh_flag_number  712311 non-null  float64
 8   ack_flag_number  712311 non-null  float64
 9   ece_flag_number  712311 non-null  float64
 10  cwr_flag_number  712311 non-null  float64
 11  ack_count        712311 non-null  int64  
 12  syn_count        712311 non-null  int64  
 13  fin_coun

In [12]:

X_columns = [
    'Header_Length', 'Protocol Type', 'Time_To_Live', 'Rate',
    'fin_flag_number', 'syn_flag_number', 'rst_flag_number',
    'psh_flag_number', 'ack_flag_number', 'ece_flag_number',
    'cwr_flag_number', 'ack_count', 'syn_count', 'fin_count',
    'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP',
    'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP',
    'IGMP', 'IPv', 'LLC', 'Tot sum', 'Min', 'Max', 'AVG',
    'Std', 'Tot size', 'IAT', 'Number', 'Variance'
]

y_column = 'Label'

df['Std'].fillna(df['Std'].mean(), inplace=True)
df['Variance'].fillna(df['Variance'].mean(), inplace=True)


print("\nHandling infinite values...")

df.replace([np.inf, -np.inf], np.nan, inplace=True)


Handling infinite values...


In [13]:

for col in X_columns:
    if df[col].isnull().any():
        mean_val = df[col].mean()
        df[col].fillna(mean_val, inplace=True)
        print(f"Imputed NaN values in column '{col}' with mean: {mean_val:.4f}")


print(f"\nUnique labels in dataset: {df[y_column].nunique()}")
print(f"Label distribution:\n{df[y_column].value_counts()}")

X = df[X_columns]
y = df[y_column]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

Imputed NaN values in column 'Rate' with mean: 28416.8064

Unique labels in dataset: 34
Label distribution:
Label
DDOS-ICMP_FLOOD            108662
DDOS-UDP_FLOOD              82011
DDOS-TCP_FLOOD              68289
DDOS-PSHACK_FLOOD           62171
DDOS-RSTFINFLOOD            61652
DDOS-SYN_FLOOD              61460
DDOS-SYNONYMOUSIP_FLOOD     54749
DOS-UDP_FLOOD               50371
DOS-TCP_FLOOD               40391
DOS-SYN_FLOOD               30620
BENIGN                      16577
MIRAI-GREETH_FLOOD          15135
MIRAI-UDPPLAIN              13342
MIRAI-GREIP_FLOOD           11187
DDOS-ICMP_FRAGMENTATION      6784
VULNERABILITYSCAN            5805
MITM-ARPSPOOFING             4590
DDOS-ACK_FRAGMENTATION       4308
DDOS-UDP_FRAGMENTATION       4264
DNS_SPOOFING                 2738
RECON-HOSTDISCOVERY          2045
RECON-OSSCAN                 1433
RECON-PORTSCAN               1251
DOS-HTTP_FLOOD               1113
DDOS-HTTP_FLOOD               416
DDOS-SLOWLORIS                354
DI

In [14]:

print("\nScaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_columns)


print("\nSelect classification type:")
print("1. Binary classification (Attack vs Benign)")
print("2. Group classification (8 classes)")
print("3. Individual classification (all unique classes)")



Scaling features...

Select classification type:
1. Binary classification (Attack vs Benign)
2. Group classification (8 classes)
3. Individual classification (all unique classes)


In [8]:
# Group classification mapping (8 classes)
def create_group_classifier_labels(labels):
    dict_7classes = {
        'DDoS-RSTFINFlood': 'DDoS',
        'DDoS-PSHACK_Flood': 'DDoS',
        'DDoS-SYN_Flood': 'DDoS',
        'DDoS-UDP_Flood': 'DDoS',
        'DDoS-TCP_Flood': 'DDoS',
        'DDoS-ICMP_Flood': 'DDoS',
        'DDoS-SynonymousIP_Flood': 'DDoS',
        'DDoS-ACK_Fragmentation': 'DDoS',
        'DDoS-UDP_Fragmentation': 'DDoS',
        'DDoS-ICMP_Fragmentation': 'DDoS',
        'DDoS-SlowLoris': 'DDoS',
        'DDoS-HTTP_Flood': 'DDoS',
        'DoS-UDP_Flood': 'DoS',
        'DoS-SYN_Flood': 'DoS',
        'DoS-TCP_Flood': 'DoS',
        'DoS-HTTP_Flood': 'DoS',
        'Mirai-greeth_flood': 'Mirai',
        'Mirai-greip_flood': 'Mirai',
        'Mirai-udpplain': 'Mirai',
        'Recon-PingSweep': 'Recon',
        'Recon-OSScan': 'Recon',
        'Recon-PortScan': 'Recon',
        'VulnerabilityScan': 'Recon',
        'Recon-HostDiscovery': 'Recon',
        'DNS_Spoofing': 'Spoofing',
        'MITM-ArpSpoofing': 'Spoofing',
        'BenignTraffic': 'Benign',
        'BrowserHijacking': 'Web',
        'Backdoor_Malware': 'Web',
        'XSS': 'Web',
        'Uploading_Attack': 'Web',
        'SqlInjection': 'Web',
        'CommandInjection': 'Web',
        'DictionaryBruteForce': 'BruteForce'
    }

    # Map labels to groups, keep original if not in mapping
    return [dict_7classes.get(label, 'Other') for label in labels]

# Apply group classification (modify this based on your preference)
classification_type = "group"  # Change to "binary" or "individual" as needed

if classification_type == "group":
    y_train_mapped = create_group_classifier_labels(y_train)
    y_test_mapped = create_group_classifier_labels(y_test)
    model_suffix = "8class"
elif classification_type == "binary":
    # Binary classification: Benign vs Attack
    y_train_mapped = ['Benign' if 'Benign' in label else 'Attack' for label in y_train]
    y_test_mapped = ['Benign' if 'Benign' in label else 'Attack' for label in y_test]
    model_suffix = "2class"
else:
    # Individual classification
    y_train_mapped = y_train
    y_test_mapped = y_test
    model_suffix = "individual"

print(f"\nUsing {classification_type} classification")
print(f"Unique classes after mapping: {len(set(y_train_mapped))}")



Using group classification
Unique classes after mapping: 2


In [9]:
# Model training and evaluation
models = {
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'RandomForest': RandomForestClassifier(random_state=42, n_estimators=100),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'MLP': MLPClassifier(random_state=42, max_iter=500)
}

# Select which models to train
models_to_train = ['LogisticRegression', 'RandomForest','AdaBoost','MLP']  # Add more as needed

results = {}

for model_name in models_to_train:
    print(f"\n{'='*50}")
    print(f"Training {model_name}...")
    print(f"{'='*50}")

    start_time = datetime.now()
    model = models[model_name]

    # Train the model
    print(f"{datetime.now()} : Fitting {model_name} model...")
    model.fit(X_train_scaled, y_train_mapped)
    print(f"{datetime.now()} : Fitting {model_name} model complete...")

    # Save the model
    model_filename = f"new-{model_name.lower()}-{model_suffix}-model.pkl"
    with open(model_filename, "wb") as f:
        pickle.dump({'model': model, 'scaler': scaler}, f)
    print(f"Model saved as {model_filename}")

    # Make predictions
    print("Making predictions...")
    y_pred = model.predict(X_test_scaled)

    # Calculate metrics
    accuracy = accuracy_score(y_test_mapped, y_pred)
    recall = recall_score(y_test_mapped, y_pred, average='macro')
    precision = precision_score(y_test_mapped, y_pred, average='macro')
    f1 = f1_score(y_test_mapped, y_pred, average='macro')

    # Store results
    results[model_name] = {
        'accuracy': accuracy,
        'recall': recall,
        'precision': precision,
        'f1': f1,
        'training_time': datetime.now() - start_time
    }

    # Print results
    print(f"\n{model_name} Results:")
    print(f"Accuracy Score: {accuracy:.6f}")
    print(f"Recall Score: {recall:.6f}")
    print(f"Precision Score: {precision:.6f}")
    print(f"F1 Score: {f1:.6f}")
    print(f"Training Time: {datetime.now() - start_time}")



Training LogisticRegression...
2025-09-19 13:23:01.731642 : Fitting LogisticRegression model...
2025-09-19 13:23:05.332368 : Fitting LogisticRegression model complete...
Model saved as new-logisticregression-8class-model.pkl
Making predictions...

LogisticRegression Results:
Accuracy Score: 0.999909
Recall Score: 0.500000
Precision Score: 0.499954
F1 Score: 0.499977
Training Time: 0:00:06.165160

Training RandomForest...
2025-09-19 13:23:07.896776 : Fitting RandomForest model...
2025-09-19 13:23:45.843840 : Fitting RandomForest model complete...
Model saved as new-randomforest-8class-model.pkl
Making predictions...

RandomForest Results:
Accuracy Score: 0.999909
Recall Score: 0.500000
Precision Score: 0.499954
F1 Score: 0.499977
Training Time: 0:00:40.798407

Training AdaBoost...
2025-09-19 13:23:48.695222 : Fitting AdaBoost model...
2025-09-19 13:24:43.599692 : Fitting AdaBoost model complete...
Model saved as new-adaboost-8class-model.pkl
Making predictions...

AdaBoost Results:
Acc

In [10]:

# Summary of all results
print(f"\n{'='*70}")
print("SUMMARY OF ALL MODELS")
print(f"{'='*70}")
print(f"{'Model':<20} {'Accuracy':<12} {'Recall':<12} {'Precision':<12} {'F1':<12}")
print(f"{'-'*70}")
for model_name, metrics in results.items():
    print(f"{model_name:<20} {metrics['accuracy']:<12.4f} {metrics['recall']:<12.4f} "
          f"{metrics['precision']:<12.4f} {metrics['f1']:<12.4f}")

print(f"\nTotal samples processed: {len(df)}")
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# Function to load and use a saved model for prediction
def load_and_predict(model_path, new_data):
    """
    Load a saved model and make predictions on new data

    Parameters:
    model_path: path to the saved pickle file
    new_data: DataFrame with the same feature columns

    Returns:
    predictions: array of predicted labels
    """
    with open(model_path, 'rb') as f:
        saved_objects = pickle.load(f)

    model = saved_objects['model']
    scaler = saved_objects['scaler']

    # Scale the new data using the saved scaler
    new_data_scaled = scaler.transform(new_data[X_columns])

    # Make predictions
    predictions = model.predict(new_data_scaled)

    return predictions

print("\nTo use a saved model for predictions:")
print("predictions = load_and_predict('model_filename.pkl', your_new_dataframe)")


SUMMARY OF ALL MODELS
Model                Accuracy     Recall       Precision    F1          
----------------------------------------------------------------------
LogisticRegression   0.9999       0.5000       0.5000       0.5000      
RandomForest         0.9999       0.5000       0.5000       0.5000      
AdaBoost             0.9999       0.5000       0.5000       0.5000      
MLP                  0.9999       0.5000       0.5000       0.5000      

Total samples processed: 712311
Training samples: 569848
Test samples: 142463

To use a saved model for predictions:
predictions = load_and_predict('model_filename.pkl', your_new_dataframe)


In [15]:
# # Improved CICIoT ML Classifier with Class Imbalance Handling

# import pandas as pd
# import numpy as np
# import pickle
# from tqdm import tqdm
# import warnings
# from datetime import datetime
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split, StratifiedKFold
# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.utils.class_weight import compute_class_weight
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.pipeline import Pipeline as ImbPipeline
# import matplotlib.pyplot as plt
# import seaborn as sns

# warnings.filterwarnings('ignore')

# # # Load your CSV file
# # CSV_FILE_PATH = 'your_file.csv'
# # df = pd.read_csv(CSV_FILE_PATH)

# print(f"Dataset shape: {df.shape}")

# # Define feature columns
# X_columns = [
#     'Header_Length', 'Protocol Type', 'Time_To_Live', 'Rate',
#     'fin_flag_number', 'syn_flag_number', 'rst_flag_number',
#     'psh_flag_number', 'ack_flag_number', 'ece_flag_number',
#     'cwr_flag_number', 'ack_count', 'syn_count', 'fin_count',
#     'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP',
#     'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP',
#     'IGMP', 'IPv', 'LLC', 'Tot sum', 'Min', 'Max', 'AVG',
#     'Std', 'Tot size', 'IAT', 'Number', 'Variance'
# ]

# y_column = 'Label'

# # Handle missing values
# df['Std'].fillna(df['Std'].mean(), inplace=True)
# df['Variance'].fillna(df['Variance'].mean(), inplace=True)

# # DETAILED CLASS DISTRIBUTION ANALYSIS
# print("\n" + "="*60)
# print("CLASS DISTRIBUTION ANALYSIS")
# print("="*60)

# print(f"\nTotal unique labels: {df[y_column].nunique()}")
# label_counts = df[y_column].value_counts()
# print(f"\nLabel distribution:")
# for label, count in label_counts.items():
#     percentage = (count / len(df)) * 100
#     print(f"{label:<25}: {count:>8} ({percentage:>6.2f}%)")

# # Check if dataset is extremely imbalanced
# majority_class_ratio = label_counts.iloc[0] / len(df)
# print(f"\nMajority class ratio: {majority_class_ratio:.4f}")

# if majority_class_ratio > 0.95:
#     print("⚠️  WARNING: Severe class imbalance detected!")
#     print("   Majority class represents >95% of data")

# # Improved classification mapping with better handling
# def create_balanced_classification(labels, classification_type="binary"):
#     """Create more balanced classification by intelligently grouping classes"""

#     if classification_type == "binary":
#         # Simple binary: Benign vs Everything else
#         binary_mapping = []
#         for label in labels:
#             if 'Benign' in str(label) or 'Normal' in str(label):
#                 binary_mapping.append('Benign')
#             else:
#                 binary_mapping.append('Attack')
#         return binary_mapping

#     elif classification_type == "3class":
#         # 3-class: Benign, DDoS/DoS, Other Attacks
#         three_class_mapping = []
#         for label in labels:
#             label_str = str(label)
#             if 'Benign' in label_str or 'Normal' in label_str:
#                 three_class_mapping.append('Benign')
#             elif any(attack in label_str for attack in ['DDoS', 'DoS']):
#                 three_class_mapping.append('DDoS_DoS')
#             else:
#                 three_class_mapping.append('Other_Attack')
#         return three_class_mapping

#     else:  # group classification
#         dict_groups = {
#             'DDoS-RSTFINFlood': 'DDoS', 'DDoS-PSHACK_Flood': 'DDoS',
#             'DDoS-SYN_Flood': 'DDoS', 'DDoS-UDP_Flood': 'DDoS',
#             'DDoS-TCP_Flood': 'DDoS', 'DDoS-ICMP_Flood': 'DDoS',
#             'DDoS-SynonymousIP_Flood': 'DDoS', 'DDoS-ACK_Fragmentation': 'DDoS',
#             'DDoS-UDP_Fragmentation': 'DDoS', 'DDoS-ICMP_Fragmentation': 'DDoS',
#             'DDoS-SlowLoris': 'DDoS', 'DDoS-HTTP_Flood': 'DDoS',
#             'DoS-UDP_Flood': 'DoS', 'DoS-SYN_Flood': 'DoS',
#             'DoS-TCP_Flood': 'DoS', 'DoS-HTTP_Flood': 'DoS',
#             'Mirai-greeth_flood': 'Mirai', 'Mirai-greip_flood': 'Mirai',
#             'Mirai-udpplain': 'Mirai',
#             'Recon-PingSweep': 'Recon', 'Recon-OSScan': 'Recon',
#             'Recon-PortScan': 'Recon', 'VulnerabilityScan': 'Recon',
#             'Recon-HostDiscovery': 'Recon',
#             'DNS_Spoofing': 'Spoofing', 'MITM-ArpSpoofing': 'Spoofing',
#             'BenignTraffic': 'Benign', 'Normal': 'Benign',
#             'BrowserHijacking': 'Web', 'Backdoor_Malware': 'Web',
#             'XSS': 'Web', 'Uploading_Attack': 'Web',
#             'SqlInjection': 'Web', 'CommandInjection': 'Web',
#             'DictionaryBruteForce': 'BruteForce'
#         }
#         return [dict_groups.get(str(label), 'Other') for label in labels]

# # Try different classification approaches
# classification_results = {}

# for class_type in ['binary', '3class']:
#     print(f"\n" + "="*50)
#     print(f"TESTING {class_type.upper()} CLASSIFICATION")
#     print("="*50)

#     # Create mapped labels
#     y_mapped = create_balanced_classification(df[y_column], class_type)

#     # Check new distribution
#     mapped_counts = pd.Series(y_mapped).value_counts()
#     print(f"\nMapped class distribution:")
#     for label, count in mapped_counts.items():
#         percentage = (count / len(y_mapped)) * 100
#         print(f"{label:<15}: {count:>8} ({percentage:>6.2f}%)")

#     # Skip if still too imbalanced
#     majority_ratio = mapped_counts.iloc[0] / len(y_mapped)
#     if majority_ratio > 0.99:
#         print(f"⚠️  Still too imbalanced ({majority_ratio:.4f}), skipping...")
#         continue

#     # Split data
#     X = df[X_columns]
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y_mapped, test_size=0.2, random_state=42, stratify=y_mapped
#     )

#     # Scale features
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)

#     # APPROACH 1: Class weights
#     print(f"\n--- APPROACH 1: Class Weights ---")
#     unique_classes = np.unique(y_train)
#     class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train)
#     class_weight_dict = dict(zip(unique_classes, class_weights))
#     print(f"Class weights: {class_weight_dict}")

#     # Test Random Forest with class weights
#     rf_weighted = RandomForestClassifier(
#         random_state=42,
#         n_estimators=100,
#         class_weight='balanced'
#     )

#     rf_weighted.fit(X_train_scaled, y_train)
#     y_pred_weighted = rf_weighted.predict(X_test_scaled)

#     print(f"\nRandom Forest with Class Weights:")
#     print(f"Accuracy: {accuracy_score(y_test, y_pred_weighted):.4f}")
#     print(f"F1 Score: {f1_score(y_test, y_pred_weighted, average='macro'):.4f}")
#     print(f"Recall: {recall_score(y_test, y_pred_weighted, average='macro'):.4f}")
#     print(f"Precision: {precision_score(y_test, y_pred_weighted, average='macro'):.4f}")

#     # APPROACH 2: SMOTE (if minority classes aren't too small)
#     min_class_size = mapped_counts.min()
#     if min_class_size >= 100:  # SMOTE needs sufficient samples
#         print(f"\n--- APPROACH 2: SMOTE Oversampling ---")

#         # Use SMOTE with Random Forest
#         smote = SMOTE(random_state=42, k_neighbors=min(5, min_class_size-1))

#         try:
#             X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

#             print(f"After SMOTE:")
#             smote_counts = pd.Series(y_train_smote).value_counts()
#             for label, count in smote_counts.items():
#                 print(f"{label}: {count}")

#             rf_smote = RandomForestClassifier(random_state=42, n_estimators=100)
#             rf_smote.fit(X_train_smote, y_train_smote)
#             y_pred_smote = rf_smote.predict(X_test_scaled)

#             print(f"\nRandom Forest with SMOTE:")
#             print(f"Accuracy: {accuracy_score(y_test, y_pred_smote):.4f}")
#             print(f"F1 Score: {f1_score(y_test, y_pred_smote, average='macro'):.4f}")
#             print(f"Recall: {recall_score(y_test, y_pred_smote, average='macro'):.4f}")
#             print(f"Precision: {precision_score(y_test, y_pred_smote, average='macro'):.4f}")

#         except Exception as e:
#             print(f"SMOTE failed: {e}")

#     # APPROACH 3: Undersampling majority class
#     print(f"\n--- APPROACH 3: Random Undersampling ---")

#     # Target size for undersampling (adjust as needed)
#     target_size = min(50000, min_class_size * 5)  # 5x the minority class or 50k max

#     undersampler = RandomUnderSampler(
#         random_state=42,
#         sampling_strategy={
#             class_name: min(count, target_size)
#             for class_name, count in mapped_counts.items()
#         }
#     )

#     X_train_under, y_train_under = undersampler.fit_resample(X_train_scaled, y_train)

#     print(f"After undersampling:")
#     under_counts = pd.Series(y_train_under).value_counts()
#     for label, count in under_counts.items():
#         print(f"{label}: {count}")

#     rf_under = RandomForestClassifier(random_state=42, n_estimators=100)
#     rf_under.fit(X_train_under, y_train_under)
#     y_pred_under = rf_under.predict(X_test_scaled)

#     print(f"\nRandom Forest with Undersampling:")
#     print(f"Accuracy: {accuracy_score(y_test, y_pred_under):.4f}")
#     print(f"F1 Score: {f1_score(y_test, y_pred_under, average='macro'):.4f}")
#     print(f"Recall: {recall_score(y_test, y_pred_under, average='macro'):.4f}")
#     print(f"Precision: {precision_score(y_test, y_pred_under, average='macro'):.4f}")

#     # Detailed classification report for best approach
#     print(f"\n--- DETAILED CLASSIFICATION REPORT (Undersampling) ---")
#     print(classification_report(y_test, y_pred_under))

#     classification_results[class_type] = {
#         'weighted': {
#             'accuracy': accuracy_score(y_test, y_pred_weighted),
#             'f1': f1_score(y_test, y_pred_weighted, average='macro'),
#             'model': rf_weighted,
#             'scaler': scaler
#         },
#         'undersampled': {
#             'accuracy': accuracy_score(y_test, y_pred_under),
#             'f1': f1_score(y_test, y_pred_under, average='macro'),
#             'model': rf_under,
#             'scaler': scaler
#         }
#     }

# # Summary and recommendations
# print(f"\n" + "="*70)
# print("RECOMMENDATIONS")
# print("="*70)

# print("\nBased on the analysis:")
# print("1. Your original poor metrics were due to severe class imbalance")
# print("2. The model was predicting only the majority class")
# print("3. Use one of the balanced approaches above")
# print("4. Consider collecting more data for minority classes")
# print("5. Monitor both accuracy AND F1/recall for imbalanced datasets")

# print(f"\nTo save the best performing model:")
# print("# Choose the best approach and save:")
# print("# with open('best_model.pkl', 'wb') as f:")
# print("#     pickle.dump({'model': best_model, 'scaler': best_scaler}, f)")

Dataset shape: (712311, 40)

CLASS DISTRIBUTION ANALYSIS

Total unique labels: 34

Label distribution:
DDOS-ICMP_FLOOD          :   108662 ( 15.25%)
DDOS-UDP_FLOOD           :    82011 ( 11.51%)
DDOS-TCP_FLOOD           :    68289 (  9.59%)
DDOS-PSHACK_FLOOD        :    62171 (  8.73%)
DDOS-RSTFINFLOOD         :    61652 (  8.66%)
DDOS-SYN_FLOOD           :    61460 (  8.63%)
DDOS-SYNONYMOUSIP_FLOOD  :    54749 (  7.69%)
DOS-UDP_FLOOD            :    50371 (  7.07%)
DOS-TCP_FLOOD            :    40391 (  5.67%)
DOS-SYN_FLOOD            :    30620 (  4.30%)
BENIGN                   :    16577 (  2.33%)
MIRAI-GREETH_FLOOD       :    15135 (  2.12%)
MIRAI-UDPPLAIN           :    13342 (  1.87%)
MIRAI-GREIP_FLOOD        :    11187 (  1.57%)
DDOS-ICMP_FRAGMENTATION  :     6784 (  0.95%)
VULNERABILITYSCAN        :     5805 (  0.81%)
MITM-ARPSPOOFING         :     4590 (  0.64%)
DDOS-ACK_FRAGMENTATION   :     4308 (  0.60%)
DDOS-UDP_FRAGMENTATION   :     4264 (  0.60%)
DNS_SPOOFING           